In [ ]:
from datetime import date

import pandas as pd

import numpy as np

import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker

In [ ]:
link_df = 'https://github.com/SaraZachi/programa-de-imuno-nacional/blob/main/Dados/Imunos%202009-2019.csv?raw=true'
dados = pd.read_csv(link_df ,encoding = 'ISO-8859-1',
                    skiprows = 3, sep=';',skipfooter=20,
                    thousands = ".", decimal =',')
dados.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.


,Unidade da Federação,072 BCG,099 Hepatite B em crianças até 30 dias,061 Rotavírus Humano,053 Meningococo C,073 Hepatite B,080 Penta,012 Pneumocócica,074 Poliomielite,100 Poliomielite 4 anos,006 Febre Amarela,096 Hepatite A,091 Pneumocócica(1º ref),092 Meningococo C (1º ref),093 Poliomielite(1º ref),021 Tríplice Viral D1,098 Tríplice Viral D2,097 Tetra Viral(SRC+VZ),075 DTP,102 DTP REF (4 e 6 anos),095 Tríplice Bacteriana(DTP)(1º ref),094 Dupla adulto e tríplice acelular gestante,003 dTpa gestante,Total
0,11 Rondônia,99.97,77.44,91.64,85.69,101.21,87.89,89.21,101.02,0.03,98.66,69.82,86.41,88.72,80.30,104.18,83.98,73.95,100.48,3.84,79.65,53.88,39.87,86.24
1,12 Acre,101.04,60.09,71.82,66.91,80.56,64.19,64.09,86.12,1.87,75.29,47.95,58.36,61.40,61.16,87.88,54.17,48.82,79.76,4.94,60.67,28.64,20.42,66.04
2,13 Amazonas,106.31,74.39,71.77,73.97,87.74,73.35,71.17,86.43,0.18,82.42,61.01,74.57,77.86,71.68,93.20,69.89,62.40,85.21,4.17,67.25,47.39,40.24,74.76
3,14 Roraima,108.15,79.26,79.12,76.03,87.01,77.28,78.81,88.79,0.40,83.36,66.25,78.17,81.25,80.99,92.44,75.17,70.88,87.32,5.31,77.28,44.48,41.19,78.37
4,15 Pará,101.15,60.20,70.29,65.26,80.49,60.53,60.49,82.29,0.03,76.23,47.04,61.69,64.88,57.58,89.89,51.81,45.06,78.37,3.29,57.81,29.93,20.42,64.43


In [ ]:
# Eliminando todos os zeros do df
dados.fillna(0, inplace=True)

# Eliminando a coluna Total 
dados = dados.drop('Total', axis=1)

# Eliminando os números dos imunos
new = dados.T
new.index = [idx[4:] for idx in new.index]
dados = new.T

# Simplificando 'Unidade da Federação' para 'UF' 
dados = dados.rename(columns={'ade da Federação':'uf'})

# Criando a coluna da região de cada estado ao lado de 'uf'
regiao_br = {'1':'Norte', '2':'Nordeste', '3':'Sudeste', '4':'Sul','5':'Centro-Oeste'}
dados['regiao'] = [regiao_br[idx[0]] for idx in dados['uf']]
dados = dados[np.roll(dados.columns,1)]

# Eliminando os números da coluna UF
dados['uf'] = [idx[3:] for idx in dados['uf']]

# Transformando 'uf' em index
dados = dados.set_index('uf')

dados.fillna(0, inplace=True)
  
dados.head()

,regiao,BCG,Hepatite B em crianças até 30 dias,Rotavírus Humano,Meningococo C,Hepatite B,Penta,Pneumocócica,Poliomielite,Poliomielite 4 anos,Febre Amarela,Hepatite A,Pneumocócica(1º ref),Meningococo C (1º ref),Poliomielite(1º ref),Tríplice Viral D1,Tríplice Viral D2,Tetra Viral(SRC+VZ),DTP,DTP REF (4 e 6 anos),Tríplice Bacteriana(DTP)(1º ref),Dupla adulto e tríplice acelular gestante,dTpa gestante
uf,,,,,,,,,,,,,,,,,,,,,,,
Rondônia,Norte,99.97,77.44,91.64,85.69,101.21,87.89,89.21,101.02,0.03,98.66,69.82,86.41,88.72,80.30,104.18,83.98,73.95,100.48,3.84,79.65,53.88,39.87
Acre,Norte,101.04,60.09,71.82,66.91,80.56,64.19,64.09,86.12,1.87,75.29,47.95,58.36,61.40,61.16,87.88,54.17,48.82,79.76,4.94,60.67,28.64,20.42
Amazonas,Norte,106.31,74.39,71.77,73.97,87.74,73.35,71.17,86.43,0.18,82.42,61.01,74.57,77.86,71.68,93.20,69.89,62.40,85.21,4.17,67.25,47.39,40.24
Roraima,Norte,108.15,79.26,79.12,76.03,87.01,77.28,78.81,88.79,0.40,83.36,66.25,78.17,81.25,80.99,92.44,75.17,70.88,87.32,5.31,77.28,44.48,41.19
Pará,Norte,101.15,60.20,70.29,65.26,80.49,60.53,60.49,82.29,0.03,76.23,47.04,61.69,64.88,57.58,89.89,51.81,45.06,78.37,3.29,57.81,29.93,20.42


In [ ]:
def new_df(dados):

    '''
    Função que recebe um conjunto de dados em forma de csv para:
        
        - simplificar a coluna 'Unidade da Federação' em 'UF';
        - criar uma coluna indicando a região da UF;
        - eliminar os números dos nomes dos estados;
        - eliminar a coluna total;
        - substituir espaços vazios por '0' no df;
        
    Variável:
        dados: dataframe com a base de dados coletados do datasus. Ex.: pd.read_csv()
    '''

    # Colocando zeros no lugar das células sem valor
    dados.fillna(0, inplace=True)

    # Eliminando a coluna Total 
    dados = dados.drop('Total', axis=1)

    # Eliminando os números dos imunos
    new = dados.T
    new.index = [idx[4:] for idx in new.index]
    dados = new.T

    # Simplificando 'Unidade da Federação' para 'UF' 
    dados = dados.rename(columns={'ade da Federação':'uf'})

    # Criando a coluna da região de cada estado ao lado de 'uf'
    regiao_br = {'1':'Norte', '2':'Nordeste', '3':'Sudeste', '4':'Sul','5':'Centro-Oeste'}
    dados['regiao'] = [regiao_br[idx[0]] for idx in dados['uf']]
    dados = dados[np.roll(dados.columns,1)]

    # Eliminando os números da coluna UF
    dados['uf'] = [idx[3:] for idx in dados['uf']]

    # Transformando 'uf' em index
    dados = dados.set_index('uf')

    dados.fillna(0, inplace=True)

    return dados

In [1]:
new_df(dados)

NameError: ignored

In [ ]:
def plot_format(dados, hue='UF', title=None, ylabel=None, xlabel=None, legend_title=None):
    '''
    Função que recebe o dataframe e plota um gráfico de linha com o eixo x com a data e eixo y os valores numéricos 
    Parâmetros:
    
        dados: dataframe do pandas, tipo=pd.DataFrame
        y: nome da variável com os dados numéricos, tipo=string
        hue: variável categórica para agrupar os dados em linhas de diferentes cores, padrão="UF", tipo=string
        title: título do gráfico, padrão=None, tipo=string
        ylabel: legenda do eixo y, padrão=None, tipo=string
        xlabel: legenda do eixo y, padrão="Ano/Mês", tipo=string
        legend_title: título da legenda, padrão=None, tipo=string
        formatter: formato dos dados numéricos, padrão= ticker.StrMethodFormatter("{x:,.2f}")(float com duas casas decimais), tipo=matplotlib.ticker
    '''
    # Estilo do gráfico
    sns.set()
    
    # Palheta de cores
    sns.set_palette(sns.color_palette("gist_rainbow", len(dados[hue].unique())))
    
    # Criando figura e eixos
    ax = plt.subplots(figsize=(12,8)) 

    #Criando o gráfico 
    sns.lineplot(hue=hue, data=dados, ax=ax)

    # Legendas e formatação
    plt.title(title, fontsize=20)
    plt.xlabel(xlabel, fontsize=15)
    plt.ylabel(ylabel, fontsize=15)
    plt.xticks(pd.date_range(start=dados['data'].unique()[5], end =dados['data'].unique()[-1], freq='6M'),dados.groupby('data').size().index[5::6].strftime('%b/%Y'),fontsize=12, rotation=90)
    plt.yticks(fontsize=12)
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles=handles[1:], labels=labels[1:], title=legend_title, bbox_to_anchor=(1, 1), fontsize=12, title_fontsize=15)
    ax.yaxis.set_major_formatter(formatter)
    
    #Mostrando o gráfico
    plt.show()